# Titanic

<p>This is a classical assigment of <a href="https://www.kaggle.com/competitions/titanic/overview">Kaggle ML competiotion</a>.</p>
<p>The assigmnt is: using machine learning to create a model that predicts which passengers survived the Titanic shipwreck.</p>

In [1]:
#pip install --upgrade pandas

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import time
from tqdm import tqdm # Для прогресс-бара выполнения кода ячейки.
from warnings import simplefilter # Сокращает сообщения Jupiter.

from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor

C:\Users\Roman\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


## 1. Data pre-processing

The Data has already been split by test and train datasets.

In [3]:
#df_gender = pd.read_csv('dataset/gender_submission.csv')
df_train = pd.read_csv('dataset/train.csv')
df_test = pd.read_csv('dataset/test.csv')

In [4]:
# Checking general statistics of the datases
display(df_train.describe())
display(df_test.describe())

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [5]:
def check_df(df):
    print('\033[1m' + 'Missed values, %:' + '\033[0m')
    for col in df.columns:
        if df[col].isna().sum() > 0:
            print(col, np.round_((df[col].isna().sum() / df.shape[0])*100))

    print('\033[1m' + '\nVarience if cathegorical features:' + '\033[0m')

    for col in df.columns:
        if col == 'Pclass' or col == 'Sex'or col == 'Parch' or col == 'Cabin':
            print(col, len(list(df[col].value_counts())))


    print('\033[1m' + '\nDuplicates:' + '\033[0m', df.duplicated().sum())
    
    display(df.sample(10))

In [6]:
check_df(df_train)

Missed values, %:
Age 20.0
Cabin 77.0
Embarked 0.0

Varience if cathegorical features:
Pclass 3
Sex 2
Parch 7
Cabin 147

Duplicates: 0


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
361,362,0,2,"del Carlo, Mr. Sebastiano",male,29.0,1,0,SC/PARIS 2167,27.7208,NaN,C
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,NaN,Q
713,714,0,3,"Larsson, Mr. August Viktor",male,29.0,0,0,7545,9.4833,NaN,S
389,390,1,2,"Lehmann, Miss. Bertha",female,17.0,0,0,SC 1748,12.0000,NaN,C
168,169,0,1,"Baumann, Mr. John D",male,NaN,0,0,PC 17318,25.9250,NaN,S
464,465,0,3,"Maisner, Mr. Simon",male,NaN,0,0,A/S 2816,8.0500,NaN,S
221,222,0,2,"Bracken, Mr. James H",male,27.0,0,0,220367,13.0000,NaN,S
777,778,1,3,"Emanuel, Miss. Virginia Ethel",female,5.0,0,0,364516,12.4750,NaN,S
801,802,1,2,"Collyer, Mrs. Harvey (Charlotte Annie Tate)",female,31.0,1,1,C.A. 31921,26.2500,NaN,S
342,343,0,2,"Collander, Mr. Erik Gustaf",male,28.0,0,0,248740,13.0000,NaN,S


In [7]:
check_df(df_test)

Missed values, %:
Age 21.0
Fare 0.0
Cabin 78.0

Varience if cathegorical features:
Pclass 3
Sex 2
Parch 8
Cabin 76

Duplicates: 0


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
223,1115,3,"Karlsson, Mr. Einar Gervasius",male,21.0,0,0,350053,7.7958,NaN,S
188,1080,3,"Sage, Miss. Ada",female,NaN,8,2,CA. 2343,69.5500,NaN,S
74,966,1,"Geiger, Miss. Amalie",female,35.0,0,0,113503,211.5000,C130,C
404,1296,1,"Frauenthal, Mr. Isaac Gerald",male,43.0,1,0,17765,27.7208,D40,C
68,960,1,"Tucker, Mr. Gilbert Milligan Jr",male,31.0,0,0,2543,28.5375,C53,C
124,1016,3,"Kennedy, Mr. John",male,NaN,0,0,368783,7.7500,NaN,Q
94,986,1,"Birnbaum, Mr. Jakob",male,25.0,0,0,13905,26.0000,NaN,C
273,1165,3,"Lennon, Miss. Mary",female,NaN,1,0,370371,15.5000,NaN,Q
236,1128,1,"Warren, Mr. Frank Manley",male,64.0,1,0,110813,75.2500,D37,C
43,935,2,"Corbett, Mrs. Walter H (Irene Colvin)",female,30.0,0,0,237249,13.0000,NaN,S


df_test contains 21% of missed valies in 'Age' column and 78% ones in 'Cabin' column.
<p>df_train contains 20% of missed valies in 'Age' column and 77% ones in 'Cabin' column.</p>

df_train contains 20% of missed valies in 'Age' column and 77% ones in 'Cabin' column.

'Age' column seems to be important for the further modelling, therethore missed valued should be replaced somehow.
We can replace them by median age of the passengers in accordance with cabin class.

In [8]:
    
df_train['Age'].isnull().sum()

177

In [9]:
for item in df_train['Pclass'].unique():
        filtered = df_train['Age'].isna() & (df_train['Pclass'] == item)
        df_train.loc[filtered, 'Age'] = df_train.loc[df_train['Pclass'] == item, 'Age'].median()
    
df_train['Age'].isnull().sum()

0

In [10]:
df_train['Embarked'] = df_train['Embarked'].fillna('S')

In [11]:
display(df_train.head(1))

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S


## 2. Preparation of the features

In [12]:
# Encoding of categorical features:

# Creationg encoder
encoder = OrdinalEncoder()

# Columns which will be encoded
oe_columns = ['Pclass','Sex', 'Age','SibSp', 'Parch', 'Embarked', 'Survived']


# Creating DataFrame with encoded features
oe_df_train = df_train[oe_columns]
oe_df_train = pd.DataFrame(encoder.fit_transform(oe_df_train),
                                columns=oe_df_train.columns, 
                                index=oe_df_train.index)
'''
# Разделим датафреймы с закодированными и незакодированными признаками
df_oe = df_train.copy()
for column in oe_columns:
    df_oe[column] = oe_df_train[column]
'''

'\n# Разделим датафреймы с закодированными и незакодированными признаками\ndf_oe = df_train.copy()\nfor column in oe_columns:\n    df_oe[column] = oe_df_train[column]\n'

In [13]:
oe_df_train

,Pclass,Sex,Age,SibSp,Parch,Embarked,Survived
0,2.0,1.0,28.0,1.0,0.0,2.0,0.0
1,0.0,0.0,51.0,1.0,0.0,0.0,1.0
2,2.0,0.0,34.0,0.0,0.0,2.0,1.0
3,0.0,0.0,47.0,1.0,0.0,2.0,1.0
4,2.0,1.0,47.0,0.0,0.0,2.0,0.0
...,...,...,...,...,...,...,...
886,1.0,1.0,35.0,0.0,0.0,2.0,0.0
887,0.0,0.0,24.0,0.0,0.0,2.0,1.0
888,2.0,0.0,31.0,1.0,2.0,2.0,0.0
889,0.0,1.0,34.0,0.0,0.0,0.0,1.0


In [14]:
# Assigning features and target datasets
features = oe_df_train.drop(['Survived'], axis=1)
target = oe_df_train.drop(['Survived'], axis=1)
print(oe_df_train.shape)
display(features.sample(10))

(891, 7)


,Pclass,Sex,Age,SibSp,Parch,Embarked
763,0.0,0.0,48.0,1.0,2.0,2.0
166,0.0,0.0,50.0,0.0,1.0,2.0
9,1.0,0.0,18.0,1.0,0.0,0.0
603,2.0,1.0,58.0,0.0,0.0,2.0
190,1.0,0.0,42.0,0.0,0.0,2.0
70,1.0,1.0,42.0,0.0,0.0,2.0
12,2.0,1.0,25.0,0.0,0.0,2.0
278,2.0,1.0,11.0,4.0,1.0,1.0
697,2.0,0.0,31.0,0.0,0.0,1.0
56,1.0,0.0,27.0,0.0,0.0,2.0


In [15]:
print(df_train['Pclass'].isnull().sum())
print(df_train['Sex'].isnull().sum())
print(df_train['SibSp'].isnull().sum())
print(df_train['Parch'].isnull().sum())
print(df_train['Fare'].isnull().sum())
print(df_train['Embarked'].isnull().sum())

0
0
0
0
0
0


## 3. Machine learning

### 3.1 Random forest

In [18]:
# Preparation of the features
RFR_model = RandomForestRegressor()
RFR_params = {'n_estimators':[3,10,15],
             'max_depth':[1,5]}

RFR_grid = RandomizedSearchCV(RFR_model,
                              param_distributions=RFR_params,
                              n_jobs=-1,
                              cv=2,
                              verbose=True,
                              scoring='neg_mean_squared_error')

In [19]:
# Learning the model
RFR_train_start = time.time()
RFR_grid.fit(features, target)
RFR_train_time = time.time() - RFR_train_start

RFR_RMSE_train = abs((RFR_grid.best_score_)) ** 0.5

print('Best parameters:', RFR_grid.best_estimator_)
print('RMSE:', RFR_RMSE_train)
print('Learning time, sec.:', np.round_(RFR_train_time))

Fitting 2 folds for each of 6 candidates, totalling 12 fits
Best parameters: RandomForestRegressor(max_depth=5, n_estimators=10)
RMSE: 0.6571543130207173
Learning time, sec.: 0.0


C:\Users\Roman\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


## 4. Estimation of the model